In [0]:
%scala
val storageAccount = "acarrascol" //nombre de la cuenta de almacenamiento
val container = "acarrascol" //nombre del contenedor
val sasKey = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-31T10:09:24Z&st=2024-12-08T02:09:24Z&spr=https&sig=%2F%2FUznJFJ458OUg%2FyT81FbLwIiv9tL1dg6uqWBq%2FbVb0%3D" //llave de conexion

val conf = "fs.azure.sas." + container + "." + storageAccount + ".blob.core.windows.net"

// Check if the directory is already mounted
if (dbutils.fs.mounts().map(_.mountPoint).contains("/mnt/" + container)) {
  dbutils.fs.unmount("/mnt/" + container)
}

// Create mount
dbutils.fs.mount(
  source = "wasbs://" + container + "@" + storageAccount + ".blob.core.windows.net",
  mountPoint = "/mnt/" + container,
  extraConfigs = Map(conf -> sasKey)
)

storageAccount: String = acarrascol
container: String = acarrascol
sasKey: String = sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-31T10:09:24Z&st=2024-12-08T02:09:24Z&spr=https&sig=%2F%2FUznJFJ458OUg%2FyT81FbLwIiv9tL1dg6uqWBq%2FbVb0%3D
conf: String = fs.azure.sas.acarrascol.acarrascol.blob.core.windows.net
res6: Boolean = true

In [0]:
%sql 
--Eliminar tabla en caso exista
DROP TABLE IF EXISTS Producto;


In [0]:

##Ingesta archivo PRODUCTO .PARQUET

producto = (spark.read.parquet("/mnt/acarrascol/proyectofinal/capaplata/Producto.parquet"))
producto.createOrReplaceTempView("s_producto")

In [0]:
## Ingesta archivo CATEGORIA.PARQUET
categoria = (spark.read.parquet("/mnt/acarrascol/proyectofinal/capaplata/Categoria.parquet"))
categoria.createOrReplaceTempView("s_categoria")

In [0]:
## Ingesta archivo SUBCATEGORIA.PARQUET
subcategoria = (spark.read.parquet("/mnt/acarrascol/proyectofinal/capaplata/SubCategoria.parquet"))
subcategoria.createOrReplaceTempView("s_subcategoria")

In [0]:
%sql

--Eliminar tabla en caso exista
DROP TABLE IF EXISTS Fact_Product;

--CRear tabla
CREATE TEMP VIEW Fact_Product
AS
SELECT P.Cod_Producto, P.Nombre AS Producto, S.Nombre AS SubCategoria, C.Nombre AS Categoria
FROM s_producto P
JOIN s_subcategoria S ON P.Cod_SubCategoria = S.Cod_SubCategoria
JOIN s_categoria C ON S.Cod_Categoria = C.Cod_Categoria

In [0]:
dfFactProducto = spark.table("Fact_Product")
dfFactProducto.write.mode("overwrite").parquet("/mnt/acarrascol/proyectofinal/capaoro/FactProducto.parquet")

In [0]:
#eliminar acceso directo
dbutils.fs.unmount("/mnt/acarrascol")

True